# Part 3: Prompt Engineering Basics

## Introduction

In this part, you'll experiment with different prompting techniques to improve the quality of responses from Large Language Models (LLMs). You'll compare zero-shot, one-shot, and few-shot prompting approaches and document which works best for different types of questions.

## Learning Objectives

- Understand different prompting techniques
- Compare zero-shot, one-shot, and few-shot prompting
- Analyze the impact of prompt design on response quality

## Setup

In [1]:
# Import necessary libraries
import requests
import json

## 1. Understanding Prompting Techniques

LLMs can be prompted in different ways to get better responses:

1. **Zero-shot prompting**: Asking the model a question directly without examples
2. **One-shot prompting**: Providing one example before asking your question
3. **Few-shot prompting**: Providing multiple examples before asking your question

## 2. Creating Prompting Templates

Your first task is to create templates for different prompting strategies.

In [2]:
# Define a question to experiment with
question = "What foods should be avoided by patients with gout?"

# Example for one-shot and few-shot prompting
example_q = "What are the symptoms of gout?"
example_a = "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."

# Examples for few-shot prompting
examples = [
    ("What are the symptoms of gout?",
     "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."),
    ("How is gout diagnosed?",
     "Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.")
]

# TODO: Create prompting templates
# Zero-shot template (just the question)
zero_shot_template = "Question: {question}\nAnswer:"

# One-shot template (one example + the question)
one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

# Few-shot template (multiple examples + the question)
few_shot_template = """Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {question}
Answer:"""

# TODO: Format the templates with your question and examples
zero_shot_prompt = zero_shot_template.format(question=question)
one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
# For few-shot, you'll need to format it with the examples list
few_shot_prompt = few_shot_template.format(examples=examples, question=question)

print("Zero-shot prompt:")
print(zero_shot_prompt)
print("\nOne-shot prompt:")
print(one_shot_prompt)
print("\nFew-shot prompt:")
print(few_shot_prompt)

Zero-shot prompt:
Question: What foods should be avoided by patients with gout?
Answer:

One-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:

Few-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:


## 3. Connecting to the LLM API

Next, implement a function to send prompts to an LLM API and get responses.

In [11]:
import requests
import os
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv()
api_key = os.getenv("HUGGINGFACE_API_KEY")


def get_llm_response(prompt, model_name="facebook/bart-large-cnn", api_key=None):
    """Get a response from the LLM based on the prompt"""
    # TODO: Implement the get_llm_response function
    if not api_key:
        raise ValueError("API key is missing. Please set HUGGINGFACE_API_KEY in your .env file.")

    url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "inputs": prompt,
        "parameters": {"max_new_tokens": 256},
        "options": {"wait_for_model": True}
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    result = response.json()

    if isinstance(result, list) and "generated_text" in result[0]:
        return result[0]["generated_text"]
    
    return result

# TODO: Test your get_llm_response function with different prompts
print("Zero-shot response:")

print(get_llm_response(zero_shot_prompt, api_key=api_key))

print("One-shot response:")
print(get_llm_response(one_shot_prompt, api_key=api_key))

print("Few-shot response:")
print(get_llm_response(few_shot_prompt, api_key=api_key))


Zero-shot response:
[{'summary_text': "Question: Can gout be cured permanently? answer: Yes. Question: Is it possible to cure gout permanently? Answer: Yes, but it's not possible to completely cure it. Questions: Is there a cure for gout? Yes, there is a cure. Are you sure?"}]
One-shot response:
[{'summary_text': 'Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe. Can gout be cured permanently? Yes, but only if the gout is treated regularly. Gout is a chronic disease that affects people of all ages.'}]
Few-shot response:
[{'summary_text': 'Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe. Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.'}]


## 4. Comparing Prompting Strategies

Now, let's compare the different prompting strategies on a set of healthcare questions.

In [4]:
# List of healthcare questions to test
questions = [
    "What foods should be avoided by patients with gout?",
    "What medications are commonly prescribed for gout?",
    "How can gout flares be prevented?",
    "Is gout related to diet?",
    "Can gout be cured permanently?"
]

# TODO: Compare the different prompting strategies on these questions
# For each question:

results = []

for question in questions:
    # Create prompts using each strategy
    zero_shot_prompt = f"Question: {question}\nAnswer:"
    
    one_shot_prompt = f"""Question: {example_q}
        Answer: {example_a}

        Question: {question}
        Answer:"""

    few_shot_prompt = f"""Question: {examples[0][0]}
        Answer: {examples[0][1]}

        Question: {examples[1][0]}
        Answer: {examples[1][1]}

        Question: {question}
        Answer:"""

    # Get responses from the LLM
    try:
        zero_shot_response = get_llm_response(zero_shot_prompt, api_key=api_key)
        one_shot_response = get_llm_response(one_shot_prompt, api_key=api_key)
        few_shot_response = get_llm_response(few_shot_prompt, api_key=api_key)
        
    except Exception as e:
        zero_shot_response = one_shot_response = few_shot_response = f"Error: {e}"

    # Store the results
    results.append({
        "question": question,
        "zero_shot": zero_shot_response,
        "one_shot": one_shot_response,
        "few_shot": few_shot_response
    })

## 5. Evaluating Responses

Create a simple evaluation function to score the responses based on the presence of expected keywords.

In [13]:
def score_response(response, keywords):
    """Score a response based on the presence of expected keywords"""
    # TODO: Implement the score_response function
    # Example implementation:
    if isinstance(response, list): # original code failed for response.lower() so had to convert into flat string
        response = " ".join(str(item) for item in response)
    elif isinstance(response, dict):
        response = str(response)
        
    response = response.lower()
    found_keywords = 0
    for keyword in keywords:
        if keyword.lower() in response:
            found_keywords += 1
    return found_keywords / len(keywords) if keywords else 0

# Expected keywords for each question
expected_keywords = {
    "What foods should be avoided by patients with gout?": 
        ["purine", "red meat", "seafood", "alcohol", "beer", "organ meats"],
    "What medications are commonly prescribed for gout?": 
        ["nsaids", "colchicine", "allopurinol", "febuxostat", "probenecid", "corticosteroids"],
    "How can gout flares be prevented?": 
        ["medication", "diet", "weight", "alcohol", "water", "exercise"],
    "Is gout related to diet?": 
        ["yes", "purine", "food", "alcohol", "seafood", "meat"],
    "Can gout be cured permanently?": 
        ["manage", "treatment", "lifestyle", "medication", "chronic"]
}

# TODO: Score the responses and calculate average scores for each strategy
strategy_scores = {
    "zero_shot": [],
    "one_shot": [],
    "few_shot": []
}

for res in results:
    question = res["question"]
    keywords = expected_keywords.get(question, [])

    zero_score = score_response(res["zero_shot"], keywords)
    one_score = score_response(res["one_shot"], keywords)
    few_score = score_response(res["few_shot"], keywords)

    strategy_scores["zero_shot"].append(zero_score)
    strategy_scores["one_shot"].append(one_score)
    strategy_scores["few_shot"].append(few_score)

# Calculate average score for each strategy
average_scores = {
    strategy: sum(scores) / len(scores) if scores else 0
    for strategy, scores in strategy_scores.items()
}
# Determine which strategy performs best overall
print("\nAverage Scores by Prompting Strategy:")
for strategy, avg_score in average_scores.items():
    print(f"{strategy}: {avg_score:.2f}")


Average Scores by Prompting Strategy:
zero_shot: 0.03
one_shot: 0.04
few_shot: 0.00


## 6. Saving Results

Save your results in a structured format for auto-grading.

In [14]:
# TODO: Save your results to results/part_3/prompting_results.txt
# The file should include:
# - Raw responses for each question and strategy
# - Scores for each question and strategy
# - Average scores for each strategy
# - The best performing strategy

# Example format:
"""
# Prompt Engineering Results

## Question: What foods should be avoided by patients with gout?

### Zero-shot response:
[response text]

### One-shot response:
[response text]

### Few-shot response:
[response text]

--------------------------------------------------

## Scores

```
question,zero_shot,one_shot,few_shot
what_foods_should,0.67,0.83,0.83
what_medications_are,0.50,0.67,0.83
how_can_gout,0.33,0.50,0.67
is_gout_related,0.80,0.80,1.00
can_gout_be,0.40,0.60,0.80

average,0.54,0.68,0.83
best_method,few_shot
```
"""

import os

# Create output directory if it doesn't exist
output_dir = "results/part_3"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "prompting_results.txt")

# Save results
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("# Prompt Engineering Results\n\n")

    # Raw responses per question
    for res in results:
        q_title = res["question"]
        f.write(f"## Question: {q_title}\n\n")
        f.write("### Zero-shot response:\n")
        f.write(str(res["zero_shot"]) + "\n\n")
        f.write("### One-shot response:\n")
        f.write(str(res["one_shot"]) + "\n\n")
        f.write("### Few-shot response:\n")
        f.write(str(res["few_shot"]) + "\n\n")
        f.write("-" * 50 + "\n\n")

    # Scores for each question and strategy
    f.write("## Scores\n\n")
    f.write("```\n")
    f.write("question,zero_shot,one_shot,few_shot\n")

    for res in results:
        question_key = res["question"].lower().replace("?", "").replace(" ", "_").replace(",", "")
        z_score = score_response(res["zero_shot"], expected_keywords[res["question"]])
        o_score = score_response(res["one_shot"], expected_keywords[res["question"]])
        f_score = score_response(res["few_shot"], expected_keywords[res["question"]])
        f.write(f"{question_key},{z_score:.2f},{o_score:.2f},{f_score:.2f}\n")

    # Average scores for each strategy
    f.write("\n")
    f.write(f"average,{average_scores['zero_shot']:.2f},{average_scores['one_shot']:.2f},{average_scores['few_shot']:.2f}\n")

    # The best performing strategy
    best_strategy = max(average_scores.items(), key=lambda x: x[1])[0]
    f.write(f"best_method,{best_strategy}\n")
    f.write("```\n")

print(f"Results saved to {output_file}")


Results saved to results/part_3/prompting_results.txt


## Progress Checkpoints

1. **Prompting Templates**:
   - [ ] Create zero-shot template
   - [ ] Create one-shot template
   - [ ] Create few-shot template
   - [ ] Format templates with questions and examples

2. **LLM API Integration**:
   - [ ] Connect to the Hugging Face API
   - [ ] Test with different prompts
   - [ ] Handle API errors

3. **Comparison and Evaluation**:
   - [ ] Compare strategies on multiple questions
   - [ ] Score responses based on keywords
   - [ ] Determine the best strategy

4. **Results and Documentation**:
   - [ ] Save results in the required format
   - [ ] Document your findings

## What to Submit

1. Your implementation in a Python script `utils/prompt_comparison.py` that:
   - Defines the prompting templates
   - Connects to the Hugging Face API
   - Compares different prompting strategies
   - Scores and evaluates the responses

2. The results of your experiments in `results/part_3/prompting_results.txt` with the format shown above

The auto-grader will check:
1. That your results file contains the required sections
2. That your scoring logic correctly identifies keyword presence
3. That you've correctly calculated average scores
4. That you've identified the best performing method